In [ ]:
!pip install segmentation_models

In [23]:
import os
import cv2
import pickle
import numpy as np
import keras
from segmentation_models import Unet
from keras.layers import ZeroPadding2D, Conv2D, Concatenate, Add, MaxPool2D, UpSampling2D, Input
from keras.models import Model
from glob import glob

In [24]:
class Generator(keras.utils.Sequence):
    def __init__(self, image_files, label_files, batch_size=8, size=None):
        self.image_files = image_files
        self.label_files = label_files
        self.batch_size = batch_size
        self.size = size
    
    def __len__(self):
        return np.ceil(len(self.image_files) / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        batch_x_paths = self.image_files[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y_paths = self.label_files[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_x = []
        batch_y = []
        for i in range(len(batch_x_paths)):
            x = cv2.imread(batch_x_paths[i], 1) / 255 if self.size is None else cv2.resize(cv2.imread(batch_x_paths[i], 1), size)
            y = cv2.imread(batch_y_paths[i], 0) / 255 if self.size is None else cv2.resize(cv2.imread(batch_x_paths[i], 1), size)
            y = np.expand_dims(y, axis=-1)
            batch_x.append(x)
            batch_y.append(y)
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y

In [3]:
image_files = []
label_files = []

image_files1 = []
label_files1 = []

for path in glob("train/*.jp*g"):
    image_files.append(path)
    label_files.append("train/" + os.path.basename(path).split('_')[0] + "_mask.png")
for path in glob("valid/*.jp*g"):
    image_files1.append(path)
    label_files1.append("valid/" + os.path.basename(path).split('_')[0] + "_mask.png")
print(image_files[0])
print(label_files[0])
print("Length:", len(image_files))

train/944589_sat.jpg
train/944589_mask.png
Length: 2242


In [4]:
batch_size = 2

In [25]:
train_generator = Generator(image_files, label_files, batch_size=8)
valid_generator = Generator(image_files1, label_files1, batch_size=8)

In [27]:
model = Unet('resnet34', input_shape=(1024,1024,3), encoder_weights='imagenet')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 1024, 1024, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 1024, 1024, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_205 (ZeroPadding (None, 1030, 1030, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 512, 512, 64) 9408        zero_padding2d_205[0][0]         
__________________________________________________________________________________________________
bn0 (Batch

In [22]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 data
1 bn_data
2 zero_padding2d_137
3 conv0
4 bn0
5 relu0
6 zero_padding2d_138
7 pooling0
8 stage1_unit1_bn1
9 stage1_unit1_relu1
10 zero_padding2d_139
11 stage1_unit1_conv1
12 stage1_unit1_bn2
13 stage1_unit1_relu2
14 zero_padding2d_140
15 stage1_unit1_conv2
16 stage1_unit1_sc
17 add_65
18 stage1_unit2_bn1
19 stage1_unit2_relu1
20 zero_padding2d_141
21 stage1_unit2_conv1
22 stage1_unit2_bn2
23 stage1_unit2_relu2
24 zero_padding2d_142
25 stage1_unit2_conv2
26 add_66
27 stage1_unit3_bn1
28 stage1_unit3_relu1
29 zero_padding2d_143
30 stage1_unit3_conv1
31 stage1_unit3_bn2
32 stage1_unit3_relu2
33 zero_padding2d_144
34 stage1_unit3_conv2
35 add_67
36 stage2_unit1_bn1
37 stage2_unit1_relu1
38 zero_padding2d_145
39 stage2_unit1_conv1
40 stage2_unit1_bn2
41 stage2_unit1_relu2
42 zero_padding2d_146
43 stage2_unit1_conv2
44 stage2_unit1_sc
45 add_68
46 stage2_unit2_bn1
47 stage2_unit2_relu1
48 zero_padding2d_147
49 stage2_unit2_conv1
50 stage2_unit2_bn2
51 stage2_unit2_relu2
52 zero_padding2

In [13]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
model.fit_generator(generator=train_generator, steps_per_epoch=len(image_files) // batch_size, epochs=5, 
                    verbose=1, validation_data=valid_generator, validation_steps=len(image_files1) // batch_size)

Epoch 1/5


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[8,128,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node concatenate_8/concat}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[metrics_1/acc/Mean_1/_4435]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[8,128,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node concatenate_8/concat}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.